# Hello Loopy: Computing a Rank-One Matrix

## Setup Code

In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import loopy as lp

from loopy.version import LOOPY_USE_LANGUAGE_VERSION_2018_2

In [2]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

Choose platform:
[0] <pyopencl.Platform 'Intel(R) OpenCL HD Graphics' at 0x3e28710>
[1] <pyopencl.Platform 'Portable Computing Language' at 0x7fb58d6f48e8>


Choice [0]: 1


Set the environment variable PYOPENCL_CTX='1' to avoid being asked again.


In [3]:
n = 1024
a = cl.clrandom.rand(queue, n, dtype=np.float32)
b = cl.clrandom.rand(queue, n, dtype=np.float32)

## The Initial Kernel

In [4]:
knl = lp.make_kernel(
    "{[i,j]: 0<=i,j<n}",
    "c[i, j] = a[i]*b[j]")

In [5]:
knl = lp.set_options(knl, write_cl=True)
evt, (mat,) = knl(queue, a=a, b=b)

/tmp/ipykernel_301648/2832970647.py:2: DirectCallUncachedWarning: TranslationUnit.__call__ will become uncached in 2024, meaning it will incur possibly substantial compilation cost with every invocation. Use TranslationUnit.executor to obtain an object that holds longer-lived caches.
  evt, (mat,) = knl(queue, a=a, b=b)


#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  for (int j = 0; j <= -1 + n; ++j)
    for (int i = 0; i <= -1 + n; ++i)
      c[n * i + j] = a[i] * b[j];
}



## Transforming kernels: Loop Splitting

Next: transform kernel. Example: Split a loop into fixed-length "chunks".

In [6]:
isplit_knl = knl
isplit_knl = lp.split_iname(isplit_knl, "i", 4)

evt, (mat,) = isplit_knl(queue, a=a, b=b)

/tmp/ipykernel_301648/2026791564.py:4: DirectCallUncachedWarning: TranslationUnit.__call__ will become uncached in 2024, meaning it will incur possibly substantial compilation cost with every invocation. Use TranslationUnit.executor to obtain an object that holds longer-lived caches.
  evt, (mat,) = isplit_knl(queue, a=a, b=b)


#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#define LOOPY_CALL_WITH_INTEGER_TYPES(MACRO_NAME) \
    MACRO_NAME(int8, char) \
    MACRO_NAME(int16, short) \
    MACRO_NAME(int32, int) \
    MACRO_NAME(int64, long)
#define LOOPY_DEFINE_FLOOR_DIV_POS_B(SUFFIX, TYPE) \
    inline TYPE loopy_floor_div_pos_b_##SUFFIX(TYPE a, TYPE b) \
    { \
        if (a<0) \
            a = a - (b-1); \
        return a/b; \
    }
LOOPY_CALL_WITH_INTEGER_TYPES(LOOPY_DEFINE_FLOOR_DIV_POS_B)
#undef LOOPY_DEFINE_FLOOR_DIV_POS_B
#undef LOOPY_CALL_WITH_INTEGER_TYPES

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  for (int j = 0; j <= -1 + n; ++j)
    for (int i_outer = 0; i_outer <= -1 + loopy_floor_div_pos_b_int32(3 + n, 4); ++i_outer)
      for (int i_inner = 0; i_inner <= ((-4 + n + -4 * i_outer >= 0) ? 3 : -1 + n + -4 * i_

Want to get rid of the conditional?

## Transforming kernels: Implementation Tags

Every loop axis ("iname") comes with an *implementation tag*.

In [7]:
isplit_knl = knl
isplit_knl = lp.assume(isplit_knl, "n mod 4 = 0")
isplit_knl = lp.split_iname(isplit_knl, "i", 4)
isplit_knl = lp.tag_inames(isplit_knl, {"i_inner": "unr"})

evt, (mat,) = isplit_knl(queue, a=a, b=b)

/tmp/ipykernel_301648/589116084.py:6: DirectCallUncachedWarning: TranslationUnit.__call__ will become uncached in 2024, meaning it will incur possibly substantial compilation cost with every invocation. Use TranslationUnit.executor to obtain an object that holds longer-lived caches.
  evt, (mat,) = isplit_knl(queue, a=a, b=b)


#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#define LOOPY_CALL_WITH_INTEGER_TYPES(MACRO_NAME) \
    MACRO_NAME(int8, char) \
    MACRO_NAME(int16, short) \
    MACRO_NAME(int32, int) \
    MACRO_NAME(int64, long)
#define LOOPY_DEFINE_FLOOR_DIV_POS_B(SUFFIX, TYPE) \
    inline TYPE loopy_floor_div_pos_b_##SUFFIX(TYPE a, TYPE b) \
    { \
        if (a<0) \
            a = a - (b-1); \
        return a/b; \
    }
LOOPY_CALL_WITH_INTEGER_TYPES(LOOPY_DEFINE_FLOOR_DIV_POS_B)
#undef LOOPY_DEFINE_FLOOR_DIV_POS_B
#undef LOOPY_CALL_WITH_INTEGER_TYPES

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  for (int j = 0; j <= -1 + n; ++j)
    for (int i_outer = 0; i_outer <= loopy_floor_div_pos_b_int32(-4 + n, 4); ++i_outer)
    {
      c[n * (4 * i_outer + 0) + j] = a[4 * i_outer] * b[j];
      c[n * (4 * i_outer + 1)

May want to influence loop ordering.

----
"Map to GPU hw axis" is an iname tag as well.

Use shortcuts for less typing:

In [8]:
split_knl = knl
split_knl = lp.split_iname(split_knl, "i", 16,
        outer_tag="g.0", inner_tag="l.0")
split_knl = lp.split_iname(split_knl, "j", 16,
        outer_tag="g.1", inner_tag="l.1")

evt, (mat,) = split_knl(queue, a=a, b=b)

/tmp/ipykernel_301648/3116107817.py:7: DirectCallUncachedWarning: TranslationUnit.__call__ will become uncached in 2024, meaning it will incur possibly substantial compilation cost with every invocation. Use TranslationUnit.executor to obtain an object that holds longer-lived caches.
  evt, (mat,) = split_knl(queue, a=a, b=b)


#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(16, 16, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  if (-1 + -16 * gid(1) + -1 * lid(1) + n >= 0 && -1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
    c[n * (16 * gid(0) + lid(0)) + 16 * gid(1) + lid(1)] = a[16 * gid(0) + lid(0)] * b[16 * gid(1) + lid(1)];
}



## Transforming kernels: Leveraging data reuse

Better! But still not much data reuse.

In [9]:
fetch1_knl = knl

fetch1_knl = lp.add_prefetch(fetch1_knl, "a", fetch_outer_inames="i")
fetch1_knl = lp.add_prefetch(fetch1_knl, "b", fetch_outer_inames="i,j")

evt, (mat,) = fetch1_knl(queue, a=a, b=b)

/tmp/ipykernel_301648/1769787273.py:6: DirectCallUncachedWarning: TranslationUnit.__call__ will become uncached in 2024, meaning it will incur possibly substantial compilation cost with every invocation. Use TranslationUnit.executor to obtain an object that holds longer-lived caches.
  evt, (mat,) = fetch1_knl(queue, a=a, b=b)


#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  float a_fetch;
  float b_fetch;

  for (int i = 0; i <= -1 + n; ++i)
  {
    a_fetch = a[i];
    for (int j = 0; j <= -1 + n; ++j)
    {
      b_fetch = b[j];
      c[n * i + j] = a_fetch * b_fetch;
    }
  }
}



But this is useless for the GPU version. (demo)

---

Would like to fetch entire "access footprint" of a loop.

In [10]:
fetch_knl = split_knl

fetch_knl = lp.add_prefetch(fetch_knl, "a", ["i_inner"], default_tag="l.auto")
fetch_knl = lp.add_prefetch(fetch_knl, "b", ["j_inner"], default_tag="l.auto")

fetch_knl = lp.add_inames_for_unused_hw_axes(fetch_knl, "id:*fetch*")
evt, (mat,) = fetch_knl(queue, a=a, b=b)

/tmp/ipykernel_301648/935037062.py:7: DirectCallUncachedWarning: TranslationUnit.__call__ will become uncached in 2024, meaning it will incur possibly substantial compilation cost with every invocation. Use TranslationUnit.executor to obtain an object that holds longer-lived caches.
  evt, (mat,) = fetch_knl(queue, a=a, b=b)
/home/andreas/src/loopy/loopy/preprocess.py:387: WriteRaceConditionWarning: in kernel loopy_kernel: instruction 'a_fetch_rule' looks invalid: it assigns to indices based on local IDs, but its temporary 'a_fetch' cannot be made local because a write race across the iname(s) 'j_inner' would emerge. (Do you need to add an extra iname to your prefetch?) (add 'write_race_local(a_fetch_rule)' to silenced_warnings kernel attribute to disable)
  warn_with_kernel(
/home/andreas/src/loopy/loopy/preprocess.py:387: WriteRaceConditionWarning: in kernel loopy_kernel: instruction 'b_fetch_rule' looks invalid: it assigns to indices based on local IDs, but its temporary 'b_fetch' c

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(16, 16, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  float a_fetch[16];
  float b_fetch[16];

  if (-1 + -16 * gid(1) + -1 * lid(1) + n >= 0)
  {
    if (-1 + -16 * gid(1) + -1 * lid(0) + n >= 0)
      b_fetch[lid(0)] = b[16 * gid(1) + lid(0)];
    if (-1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
      a_fetch[lid(0)] = a[16 * gid(0) + lid(0)];
    if (-1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
      c[n * (16 * gid(0) + lid(0)) + 16 * gid(1) + lid(1)] = a_fetch[lid(0)] * b_fetch[lid(1)];
  }
}



## Transforming kernels: Eliminating Conditionals

All those conditionals take time to evaluate!

In [11]:
sfetch_knl = knl
sfetch_knl = lp.split_iname(sfetch_knl, "i", 16,
        outer_tag="g.0", inner_tag="l.0", slabs=(0,1))
sfetch_knl = lp.split_iname(sfetch_knl, "j", 16,
        outer_tag="g.1", inner_tag="l.1", slabs=(0,1))

sfetch_knl = lp.add_prefetch(sfetch_knl, "a", ["i_inner"], default_tag="l.auto")
sfetch_knl = lp.add_prefetch(sfetch_knl, "b", ["j_inner"], default_tag="l.auto")
sfetch_knl = lp.add_inames_for_unused_hw_axes(sfetch_knl, "id:*fetch*")

evt, (mat,) = sfetch_knl(queue, a=a, b=b)

/tmp/ipykernel_301648/441645651.py:11: DirectCallUncachedWarning: TranslationUnit.__call__ will become uncached in 2024, meaning it will incur possibly substantial compilation cost with every invocation. Use TranslationUnit.executor to obtain an object that holds longer-lived caches.
  evt, (mat,) = sfetch_knl(queue, a=a, b=b)


#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(16, 16, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  float a_fetch[16];
  float b_fetch[16];

  /* bulk slab for 'j_outer' */
  /* bulk slab for 'i_outer' */
  if (-17 + -16 * gid(1) + n >= 0 && -17 + -16 * gid(0) + n >= 0)
  {
    b_fetch[lid(0)] = b[16 * gid(1) + lid(0)];
    a_fetch[lid(0)] = a[16 * gid(0) + lid(0)];
    c[n * (16 * gid(0) + lid(0)) + 16 * gid(1) + lid(1)] = a_fetch[lid(0)] * b_fetch[lid(1)];
  }
  /* final slab for 'i_outer' */
  if (16 + 16 * gid(0) + -1 * n >= 0 && -17 + -16 * gid(1) + n >= 0)
  {
    b_fetch[lid(0)] = b[16 * gid(1) + lid(0)];
    if (-1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
      a_fetch[lid(0)] = a[16 * gid(0) + lid(0)];
    if (-1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
      c[n * (16 * gid(0) + lid(0)) + 16 * gid(1) +